In [6]:
import numpy as np
import pandas as pd
from math import sin, asin, cos, radians, fabs, sqrt

EARTH_RADIUS = 6371      # 地球平均半径大约6371km


def hav(theta):
    s = sin(theta / 2)
    return s * s


def get_distance_hav(lat0, lng0, lat1, lng1):
    # 用haversine公式计算球面两点间的距离
    # 经纬度转换成弧度
    lat0 = radians(lat0)
    lat1 = radians(lat1)
    lng0 = radians(lng0)
    lng1 = radians(lng1)
    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    distance = 2 * EARTH_RADIUS * asin(sqrt(h))      # km
    return distance

def gaussian_kernel(lat0, lng0, lat1, lng1, sigma):
    distance_hav = get_distance_hav(lat0, lng0, lat1, lng1)
    distance = np.linalg.norm(distance_hav)
    print(distance)
    similarity = np.exp(-distance**2 / (2* (sigma**2)))
    print(similarity)
    print('\n')
    return similarity

def compute_similarity_matrix(X,sigma):
    n = X.shape[0]
    S = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            lat0=X[i,1]
            lng0=X[i,0]
            lat1=X[j,1]
            lng1=X[j,0]
            print(lat0)
            print(lng0)
            print(lat1)
            print(lng1)
            S[i,j] = gaussian_kernel(lat0, lng0, lat1, lng1, sigma)
    return S

In [ ]:
df=pd.DataFrame(pd.read_excel(r'D:\myJupyter\paper1\Scene-GCN-Tensorflow\Scene-GCNdata\Gauss03.xlsx',usecols=['longitude','latitude']))
df=df.values
print(df)

In [ ]:
X = df
distance=[]
n = X.shape[0]
for i in range(n):
    for j in range(n):
        lat0=X[i,1]
        lng0=X[i,0]
        lat1=X[j,1]
        lng1=X[j,0]
        distance.append(get_distance_hav(lat0, lng0, lat1, lng1))
print(distance)
sigma=np.std(distance)
sigma

In [ ]:
X = df
S = compute_similarity_matrix(X, sigma)
S

In [ ]:
np.fill_diagonal(S,0)
S[S == 1] = 0
S

In [14]:
df = pd.DataFrame(S)
df.to_excel(r'D:\myJupyter\paper1\Scene-GCN-Tensorflow\Scene-GCNdata\3611817550_adj.xlsx')